In [1]:
from langame_client import LangameClient
from pprint import pprint
from firebase_admin import firestore

In [2]:
langame_client = LangameClient()

In [29]:
# TODO: use protobuf
p = {}
p["type"] = "TopicGeneralist"
p["template"] = """This is a conversation starter that leads to extremely profound conversations between people about "[TOPIC]".
Starter: What do you think of the theory that animals have emotions?
Starter: What is the role of philosophy in human society?
Starter: How does one recognize truth and knowledge? What are the ways to acquire it?
Starter:"""
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": "davinci",
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0.1,
            "presencePenalty": 0.1,
            "stop": ["\\n", "Starter:"],
        }
    }
}
p_col = langame_client._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 7, 16, 15, 49, 26, 906587, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x7f4c13177610>)


In [17]:
p = {}
p["type"] = "TopicGeneralistFineTuned"
p["template"] = """[TOPIC].
Starter:"""
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": "curie:ft-beaumont-2021-07-18-16-25-55",
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0.1,
            "presencePenalty": 0.1,
            "stop": ["\\n", "Starter:"],
        }
    }
}
p_col = langame_client._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 7, 19, 13, 20, 20, 938197, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x7f613317ff40>)


In [7]:
langame_client.prompt_to_meme(topic_with_emoji = 
    ("transhumanism", "🤔🧠💭📚📖📐🧬🤖🦾🦿"),
)

calling openai with {'id': 'SiWJjmgmA9LPx9yGvyQt', 'prompt': 'transhumanism.\nStarter:', 'parameters': {'model': 'curie:ft-beaumont-2021-07-18-16-25-55', 'topP': 1, 'frequencyPenalty': 0.1, 'maxTokens': 150, 'presencePenalty': 0.1, 'stop': ['\\n', 'Starter:', '?'], 'temperature': 0.7}}


(DatetimeWithNanoseconds(2021, 7, 19, 13, 24, 4, 207679, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f3d646e32b0>)

In [39]:
langame_client.prompt_to_meme(topic_with_emoji = 
    ("politic", "🤔💭📜🏛👨👩⚖🗽"),
)

calling openai with {'id': '4IpLQkq4VwnV4M3HheIg', 'prompt': 'It is a question that leads to extremely profound conversations between people about "politic":\n1.', 'parameters': {'model': 'davinci', 'topP': 1, 'stop': ['\\n', '2.'], 'frequencyPenalty': 0.1, 'presencePenalty': 0.1, 'maxTokens': 150, 'temperature': 0.7}}


(DatetimeWithNanoseconds(2021, 7, 16, 15, 50, 58, 292063, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f4c13177250>)

In [ ]:
langame_client.prompt_to_meme(topic_with_emoji = 
    ("philosophy", "🤔🧠💭📚📖"),
)

In [13]:
langame_client.prompt_to_meme(topic_with_emoji = 
    ("mind", "🤔🧠💭📚📖"),
)

calling openai with {'id': 'SiWJjmgmA9LPx9yGvyQt', 'prompt': 'mind.\nStarter:', 'parameters': {'model': 'curie:ft-beaumont-2021-07-18-16-25-55', 'temperature': 0.7, 'frequencyPenalty': 0.1, 'presencePenalty': 0.1, 'maxTokens': 150, 'topP': 1, 'stop': ['\\n', 'Starter:', '?']}}


(DatetimeWithNanoseconds(2021, 7, 19, 14, 10, 53, 111519, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f3d080bcdf0>)

In [12]:
for t in langame_client._firestore_client.collection_group(u"tags")\
    .where(u"topic.content", u"in", ["philosophy"])\
    .order_by(u"createdAt", direction=firestore.Query.DESCENDING)\
    .limit(10)\
    .stream():
    pprint(t.to_dict())
    pprint(t.reference.parent.parent.get().to_dict())

{'createdAt': DatetimeWithNanoseconds(2021, 6, 12, 15, 49, 16, 439000, tzinfo=<UTC>),
 'topic': {'content': 'philosophy', 'emojis': ['🤔', '🧠', '💭', '📚', '📖']}}
{'content': 'What is your biggest fear and how do you deal with it?',
 'createdAt': DatetimeWithNanoseconds(2021, 6, 12, 15, 49, 16, 270000, tzinfo=<UTC>)}
{'createdAt': DatetimeWithNanoseconds(2021, 6, 12, 15, 49, 16, 439000, tzinfo=<UTC>),
 'topic': {'content': 'philosophy', 'emojis': ['🤔', '🧠', '💭', '📚', '📖']}}
{'content': 'What is a book that changed your life for good?',
 'createdAt': DatetimeWithNanoseconds(2021, 6, 12, 15, 49, 16, 270000, tzinfo=<UTC>)}
{'createdAt': DatetimeWithNanoseconds(2021, 6, 12, 15, 49, 16, 439000, tzinfo=<UTC>),
 'topic': {'content': 'philosophy', 'emojis': ['🤔', '🧠', '💭', '📚', '📖']}}
{'content': 'What are the things that make you feel alive?',
 'createdAt': DatetimeWithNanoseconds(2021, 6, 12, 15, 49, 16, 270000, tzinfo=<UTC>)}
{'createdAt': DatetimeWithNanoseconds(2021, 6, 12, 15, 49, 5, 578000

In [16]:
# TODO: use protobuf
p = {}
p["type"] = "IceBreakerGeneralist"
p["template"] = """This is conversation starters that leads to extremely personal discussions between people.
Starter: What is an interesting fact that few people know about you?
Starter: What is something that most people take for granted, but has a deeper meaning to you?
Starter:"""
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": "davinci",
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0.1,
            "presencePenalty": 0.1,
            "stop": ["\\n", "Starter:"],
        }
    }
}
p_col = langame_client._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 7, 16, 15, 34, 18, 122873, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x7f4c131a0fd0>)


In [28]:
langame_client.prompt_to_meme_ice_breaker()

calling openai with {'id': 'oCccJ82pVND4znQx3zM5', 'prompt': 'This is conversation starters that leads to extremely personal discussions between people.\nStarter: What is an interesting fact that few people know about you?\nStarter: What is something that most people take for granted, but has a deeper meaning to you?\nStarter:', 'parameters': {'temperature': 0.7, 'maxTokens': 150, 'frequencyPenalty': 0.1, 'model': 'davinci', 'presencePenalty': 0.1, 'stop': ['\\n', 'Starter:'], 'topP': 1}}


(DatetimeWithNanoseconds(2021, 7, 16, 15, 35, 59, 312111, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f4c131a0dc0>)

In [6]:
for e in langame_client._firestore_client.collection("prompts")\
        .limit(5)\
        .stream():
    pprint(e.to_dict())

{'template': 'It is a question that leads to extremely profound conversations '
             'between people about "[TOPIC]":\n'
             '1.',
 'type': 'TopicGeneralist'}
{'template': 'Provide me a conversation starter that brings extremely deeply '
             'personal conversations between people, thus breaking the ice.\n'
             'Starter:',
 'type': 'IceBreakerGeneralist'}
{'template': 'This is a question that leads to extremely profound '
             'conversations between people and allow them to break the ice:\n'
             '1.',
 'type': 'IceBreakerGeneralist'}


In [7]:

completions = []
topics = ["philosophy", "physic", "mathematic", "biology", "transhumanism", "artificial intelligence", "book"]
for t in langame_client._firestore_client.collection_group(u"tags")\
    .where(u"topic.content", u"in", topics)\
    .order_by(u"createdAt", direction=firestore.Query.DESCENDING)\
    .stream():
    completions.append({"topic": t.to_dict()["topic"]["content"], "meme": t.reference.parent.parent.get().to_dict()["content"]})
pprint(len(completions))


171


In [8]:
#{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
def get_prompt(topic):
    return f"""This is a conversation starter that leads to extremely profound conversations between people about "{topic}".
Starter:"""

dataset = [{"prompt": get_prompt(e["topic"]), "completion": f" {e['meme']}"} for e in completions]

In [9]:
import json
with open('output.jsonl', 'w') as outfile:
    for entry in dataset:
        json.dump(entry, outfile)
        outfile.write('\n')

In [18]:
!openai tools fine_tunes.prepare_data -f output.jsonl

Analyzing...

- Your file contains 56 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples

ERROR in common_suffix validator: All prompts are identical: `This is a conversation starter that leads to extremely profound conversations between people about "philosophy".
Starter:`
Consider leaving the prompts blank if you want to do open-ended generation, otherwise ensure prompts are different

Aborting...

In [4]:
import openai
openai.File.list()

<OpenAIObject list at 0x7f6134b52090> JSON: {
  "data": [
    {
      "bytes": 53872,
      "created_at": 1626524028,
      "filename": "output.jsonl",
      "id": "file-xiORWPq4g2IsFNYD7ZVPLVB3",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    },
    {
      "bytes": 9498,
      "created_at": 1626625558,
      "filename": "compiled_results.csv",
      "id": "file-DQCtfogXgXBDNULRLSOx8kI9",
      "object": "file",
      "purpose": "fine-tune-results",
      "status": "processed",
      "status_details": null
    }
  ],
  "object": "list"
}

In [14]:
openai.File.create(
  file=open("output.jsonl"),
  purpose="fine-tune"
)

Upload progress: 100%|██████████| 54.1k/54.1k [00:00<00:00, 5.92Mit/s]


<File file id=file-xiORWPq4g2IsFNYD7ZVPLVB3 at 0x7f46cfdf75e0> JSON: {
  "bytes": 53872,
  "created_at": 1626524028,
  "filename": "output.jsonl",
  "id": "file-xiORWPq4g2IsFNYD7ZVPLVB3",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [16]:
files = openai.File.list()
files.get("data")

[<File file id=file-xiORWPq4g2IsFNYD7ZVPLVB3 at 0x7f46cc5692c0> JSON: {
   "bytes": 53872,
   "created_at": 1626524028,
   "filename": "output.jsonl",
   "id": "file-xiORWPq4g2IsFNYD7ZVPLVB3",
   "object": "file",
   "purpose": "fine-tune",
   "status": "processed",
   "status_details": null
 }]

In [21]:
openai.FineTune.create(
    training_file=files["data"][0]["id"],
    model="curie"
)

<FineTune fine-tune id=ft-oFMt7ZUOQWYPR3MiIjxqd6Ht at 0x7f4709230720> JSON: {
  "created_at": 1626524337,
  "events": [
    {
      "created_at": 1626524337,
      "level": "info",
      "message": "Created fine-tune: ft-oFMt7ZUOQWYPR3MiIjxqd6Ht",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": 4,
    "learning_rate_multiplier": 0.1,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1,
    "use_packing": null
  },
  "id": "ft-oFMt7ZUOQWYPR3MiIjxqd6Ht",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-6iVquBDkjuvNsgZOGPJsaRAs",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 53872,
      "created_at": 1626524028,
      "filename": "output.jsonl",
      "id": "file-xiORWPq4g2IsFNYD7ZVPLVB3",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1626524337,
  "user_id": "user-fs8Zlgenlu

In [5]:
model = openai.FineTune.list()["data"][0]["fine_tuned_model"]
pprint(model)

'curie:ft-beaumont-2021-07-18-16-25-55'


In [16]:
openai.Completion.create(
  model=model,
  prompt="""Physics.
Starter:""",
  max_tokens=100,
  stop=["\n"]
)

<OpenAIObject text_completion id=cmpl-3NNPbdSWZcGOMhOMmPOW9Cas4vUgk at 0x7f6133162f40> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " What happens if you drop an object from an aeroplane? (If you do not know the answer to this question, then you will not be able to progress with the conversation.) The faster an object falls, the faster it will reach the ground. This means we could travel at aeroplane speed and nothing would happen until we reached the ground but with gravity messing up our path we would end up millions of miles away from where we began. This effect is called 'gravity'. If we could"
    }
  ],
  "created": 1626700551,
  "id": "cmpl-3NNPbdSWZcGOMhOMmPOW9Cas4vUgk",
  "model": "curie:ft-beaumont-2021-07-18-16-25-55",
  "object": "text_completion"
}